In [29]:
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime

Підключення до БД

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client['online_store']

items = db['items']
orders = db['orders']

Створіть декілька товарів з різним набором властивостей Phone/TV/Smart Watch/

In [3]:
item1 = {
    "category": "Phone",
    "model": "iPhone 12",
    "producer": "Apple",
    "price": 900
}

item2 = {
    "category": "TV",
    "model": "OLED65",
    "producer": "LG",
    "price": 1200
}

item3 = {
    "category": "Smart Watch",
    "model": "Galaxy Watch",
    "producer": "Samsung",
    "price": 300
}

items.insert_many([item1, item2, item3])

InsertManyResult([ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69ca'), ObjectId('66334b7bf9ec9c8290cd69cb')], acknowledged=True)

Напишіть запит, який виводить усі товари (відображення у JSON)

In [4]:
all_items = list(items.find())
print("Всі товари:")
for item in all_items:
    print(item)

Всі товари:
{'_id': ObjectId('66334b7bf9ec9c8290cd69c9'), 'category': 'Phone', 'model': 'iPhone 12', 'producer': 'Apple', 'price': 900}
{'_id': ObjectId('66334b7bf9ec9c8290cd69ca'), 'category': 'TV', 'model': 'OLED65', 'producer': 'LG', 'price': 1200}
{'_id': ObjectId('66334b7bf9ec9c8290cd69cb'), 'category': 'Smart Watch', 'model': 'Galaxy Watch', 'producer': 'Samsung', 'price': 300}


Підрахуйте скільки товарів у певної категорії

In [5]:
category_count = items.count_documents({"category": "Phone"})
print(f"Кількість товарів у категорії Phone: {category_count}")

Кількість товарів у категорії Phone: 1


Підрахуйте скільки є різних категорій товарів


In [6]:
distinct_categories = items.distinct("category")
print(f"Різні категорії товарів: {distinct_categories}")

Різні категорії товарів: ['Phone', 'Smart Watch', 'TV']


Виведіть список всіх виробників товарів без повторів

In [7]:
distinct_producers = items.distinct("producer")
print(f"Різні виробники: {distinct_producers}")

Різні виробники: ['Apple', 'LG', 'Samsung']


Напишіть запити, які вибирають товари за різними критеріям і їх сукупності:

Категорія та ціна (в проміжку) - конструкція $and

In [8]:
filtered_items = list(items.find({"$and": [{"category": "Phone"}, {"price": {"$gt": 500, "$lt": 1000}}]}))
print("Товари категорії 'Phone' з ціною між 500 та 1000:")
for item in filtered_items:
    print(item)

Товари категорії 'Phone' з ціною між 500 та 1000:
{'_id': ObjectId('66334b7bf9ec9c8290cd69c9'), 'category': 'Phone', 'model': 'iPhone 12', 'producer': 'Apple', 'price': 900}


Модель чи одна чи інша ($or)

In [9]:
filtered_items = list(items.find({"$or": [{"model": "iPhone 12"}, {"model": "OLED65"}]}))
print("Товари моделей 'iPhone 12' або 'OLED65':")
for item in filtered_items:
    print(item)

Товари моделей 'iPhone 12' або 'OLED65':
{'_id': ObjectId('66334b7bf9ec9c8290cd69c9'), 'category': 'Phone', 'model': 'iPhone 12', 'producer': 'Apple', 'price': 900}
{'_id': ObjectId('66334b7bf9ec9c8290cd69ca'), 'category': 'TV', 'model': 'OLED65', 'producer': 'LG', 'price': 1200}


Виробники з переліку ($in)

In [10]:
filtered_items = list(items.find({"producer": {"$in": ["Apple", "Samsung"]}}))
print("Товари виробників 'Apple' або 'Samsung':")
for item in filtered_items:
    print(item)

Товари виробників 'Apple' або 'Samsung':
{'_id': ObjectId('66334b7bf9ec9c8290cd69c9'), 'category': 'Phone', 'model': 'iPhone 12', 'producer': 'Apple', 'price': 900}
{'_id': ObjectId('66334b7bf9ec9c8290cd69cb'), 'category': 'Smart Watch', 'model': 'Galaxy Watch', 'producer': 'Samsung', 'price': 300}


Оновіть певні товари, змінивши існуючі значення і додайте нові властивості (характеристики) усім товарам за певним критерієм

In [11]:
# Наприклад, додати нові властивості та збільшити вартість
result = items.update_many(
    {"category": "Phone"},
    {"$set": {"warranty": "2 years"}, "$inc": {"price": 50}}
)

print(f"Кількість оновлених документів: {result.modified_count}")


Кількість оновлених документів: 1


Знайдіть товари у яких є (присутнє поле) певні властивості

In [12]:
# Припустимо, що ми хочемо знайти товари, які містять поле "warranty".
field_name = "warranty"

# Виконання запиту до колекції items, щоб знайти товари, у яких є поле "warranty"
items_with_field = list(items.find({field_name: {"$exists": True}}))

# Вивести товари, які містять поле "warranty"
print(f"Товари, які містять поле '{field_name}':")
for item in items_with_field:
    print(item)


Товари, які містять поле 'warranty':
{'_id': ObjectId('66334b7bf9ec9c8290cd69c9'), 'category': 'Phone', 'model': 'iPhone 12', 'producer': 'Apple', 'price': 950, 'warranty': '2 years'}


Для знайдених товарів збільшіть їх вартість на певну суму

In [13]:
# Сума, на яку потрібно збільшити вартість
increase_amount = 50  # Збільшення на 50 одиниць

# Виконання запиту до колекції items для збільшення вартості на певну суму
result = items.update_many(
    {field_name: {"$exists": True}},  # Фільтр: товари з певним полем
    {"$inc": {"price": increase_amount}}  # Операція збільшення вартості
)

# Виведення кількості оновлених документів
print(f"Кількість оновлених товарів: {result.modified_count}")

Кількість оновлених товарів: 1


Створіть кілька замовлень з різними наборами товарів, але так щоб один з товарів був у декількох замовленнях

In [59]:
# Додавання замовлень з товарами у вигляді references
order1 = {
    "order_number": 1001,
    "date": "2024-04-25",
    "total_sum": item1["price"] + item2["price"],
    "customer": {
        "name": "Andrii",
        "surname": "Rodinov",
        "phones": [9876543, 1234567],
        "address": "PTI, Peremohy 37, Kyiv, UA"
    },
    "payment": {
        "card_owner": "Andrii Rodionov",
        "card_id": 12345678
    },
    "items_id": [item1["_id"], item2["_id"]]
}

order2 = {
    "order_number": 1002,
    "date": "2023-04-25",
    "total_sum": item1["price"] + item3["price"],
    "customer": {
        "name": "Ivan",
        "surname": "Shevchenko",
        "phones": [2345678],
        "address": "PTI, Peremohy 37, Kyiv, UA"
    },
    "payment": {
        "card_owner": "Ivan Shevchenko",
        "card_id": 87654321
    },
    "items_id": [item1["_id"], item3["_id"]]
}

# Додавання замовлень до колекції orders
orders.insert_many([order1, order2])

InsertManyResult([ObjectId('66335b0cf9ec9c8290cd69d6'), ObjectId('66335b0cf9ec9c8290cd69d7')], acknowledged=True)

Виведіть всі замовлення

In [60]:
# Вивести всі замовлення
all_orders = list(orders.find())
print("Всі замовлення:")
for order in all_orders:
    print(order)

Всі замовлення:
{'_id': ObjectId('66335b0cf9ec9c8290cd69d6'), 'order_number': 1001, 'date': '2024-04-25', 'total_sum': 2100, 'customer': {'name': 'Andrii', 'surname': 'Rodinov', 'phones': [9876543, 1234567], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Andrii Rodionov', 'card_id': 12345678}, 'items_id': [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69ca')]}
{'_id': ObjectId('66335b0cf9ec9c8290cd69d7'), 'order_number': 1002, 'date': '2023-04-25', 'total_sum': 1200, 'customer': {'name': 'Ivan', 'surname': 'Shevchenko', 'phones': [2345678], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Ivan Shevchenko', 'card_id': 87654321}, 'items_id': [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69cb')]}


Виведіть замовлення з вартістю більше певного значення

In [36]:
high_value_orders = list(orders.find({"total_sum": {"$gt": 1000}}))
print("Замовлення з вартістю більше 1000:")
for order in high_value_orders:
    print(order)

Замовлення з вартістю більше 1000:
{'_id': ObjectId('66334ca0f9ec9c8290cd69cc'), 'order_number': 1001, 'date': '2024-04-25', 'total_sum': 2400, 'customer': {'name': 'Andrii', 'surname': 'Rodinov', 'phones': [9876543, 1234567], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Andrii Rodionov', 'card_id': 12345678}, 'items_id': [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69ca'), ObjectId('66334ff8f9ec9c8290cd69ce')]}
{'_id': ObjectId('66334ca0f9ec9c8290cd69cd'), 'order_number': 1002, 'date': '2024-04-25', 'total_sum': 1500, 'customer': {'name': 'Ivan', 'surname': 'Shevchenko', 'phones': [2345678], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Ivan Shevchenko', 'card_id': 87654321}, 'items_id': [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69cb'), ObjectId('66334ff8f9ec9c8290cd69ce')]}


Знайдіть замовлення зроблені одним замовником

In [17]:
# Ідентифікаційні дані замовника (наприклад, ім'я та прізвище)
customer_name = "Andrii"
customer_surname = "Rodinov"

# Запит для знаходження всіх замовлень, зроблених одним замовником
customer_orders = list(orders.find({
    "customer.name": customer_name,
    "customer.surname": customer_surname
}))

# Вивід результатів
print(f"Замовлення, зроблені замовником {customer_name} {customer_surname}:")
for order in customer_orders:
    print(order)

Замовлення, зроблені замовником Andrii Rodinov:
{'_id': ObjectId('66334ca0f9ec9c8290cd69cc'), 'order_number': 1001, 'date': '2024-04-25', 'total_sum': 1200, 'customer': {'name': 'Andrii', 'surname': 'Rodinov', 'phones': [9876543, 1234567], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Andrii Rodionov', 'card_id': 12345678}, 'items_id': [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69ca')]}


Знайдіть всі замовлення з певним товаром (товарами) (шукати можна по ObjectId)

In [18]:
# Вказаний ObjectId товару або товарів, які потрібно знайти у замовленнях
item_ids_to_find = [item1["_id"], item2["_id"]]

# Запит для знаходження всіх замовлень, які містять певний товар (товари)
orders_with_items = list(orders.find({"items_id": {"$in": item_ids_to_find}}))

# Вивід результатів
print(f"Замовлення, які містять товари з ObjectId {item_ids_to_find}:")
for order in orders_with_items:
    print(order)

Замовлення, які містять товари з ObjectId [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69ca')]:
{'_id': ObjectId('66334ca0f9ec9c8290cd69cc'), 'order_number': 1001, 'date': '2024-04-25', 'total_sum': 1200, 'customer': {'name': 'Andrii', 'surname': 'Rodinov', 'phones': [9876543, 1234567], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Andrii Rodionov', 'card_id': 12345678}, 'items_id': [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69ca')]}
{'_id': ObjectId('66334ca0f9ec9c8290cd69cd'), 'order_number': 1002, 'date': '2024-04-25', 'total_sum': 300, 'customer': {'name': 'Ivan', 'surname': 'Shevchenko', 'phones': [2345678], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 'payment': {'card_owner': 'Ivan Shevchenko', 'card_id': 87654321}, 'items_id': [ObjectId('66334b7bf9ec9c8290cd69c9'), ObjectId('66334b7bf9ec9c8290cd69cb')]}


Додайте в усі замовлення з певним товаром ще один товар і збільште існуючу вартість замовлення на деяке значення Х

In [19]:
item4 = {
    "category": "TV",
    "model": "Sony Bravia",
    "producer": "Sony",
    "price": 1200
}

items.insert_many([item4])

InsertManyResult([ObjectId('66334ff8f9ec9c8290cd69ce')], acknowledged=True)

In [20]:
target_item_id = item1["_id"]

# ObjectId товару, який потрібно додати до замовлень
new_item_id = item4["_id"]

# Сума, на яку потрібно збільшити існуючу вартість замовлення
increase_amount = item4["price"] # Збільшення на 100 одиниць

# Знайти всі замовлення, які містять певний товар
orders_with_target_item = orders.find({"items_id": target_item_id})

# Оновлення кожного знайденого замовлення
for order in orders_with_target_item:
    # Оновлення списку товарів та загальної суми замовлення
    result = orders.update_one(
        {"_id": order["_id"]},
        {
            # Додати новий товар до списку товарів
            "$addToSet": {"items_id": new_item_id},
            # Збільшити існуючу вартість замовлення
            "$inc": {"total_sum": increase_amount}
        }
    )
    print(f"Замовлення з id {order['_id']} оновлене: {result.modified_count}")

print("Усі замовлення з певним товаром оновлені.")

Замовлення з id 66334ca0f9ec9c8290cd69cc оновлене: 1
Замовлення з id 66334ca0f9ec9c8290cd69cd оновлене: 1
Усі замовлення з певним товаром оновлені.


Виведіть кількість товарів в певному замовленні

In [22]:
# Унікальний ідентифікатор замовлення (ObjectId)
order_id = order1["_id"]

# Знаходимо замовлення за його ObjectId
order = orders.find_one({"_id": order_id})

# Перевірка, чи замовлення знайдено
if order:
    # Підрахунок кількості товарів у замовленні (довжина списку items_id)
    num_items = len(order.get("items_id", []))
    print(f"Кількість товарів у замовленні з id {order_id}: {num_items}")
else:
    print(f"Замовлення з id {order_id} не знайдено.")

Кількість товарів у замовленні з id 66334ca0f9ec9c8290cd69cc: 3


Виведіть тільки інформацію про кастомера і номери кредитної карт, для замовлень вартість яких перевищує певну суму

In [28]:
# Визначення порогової суми
threshold_sum = 1500 # Наприклад, 1000 одиниць

# Виконання запиту до колекції orders з фільтром та проекцією
# Фільтр: замовлення, вартість яких перевищує threshold_sum
# Проекція: включаємо поля "customer" та "payment.cardId"
result = orders.find(
    {"total_sum": {"$gt": threshold_sum}},  # Фільтр за вартістю замовлення
    {
        "customer": 1,  # Включаємо інформацію про замовника
        "payment": 1  # Включаємо номер кредитної карти
    }
)

# Вивід результатів
print(f"Замовлення, вартість яких перевищує {threshold_sum} одиниць:")
for order in result:
    # Виводимо інформацію про замовника та номер кредитної картки, якщо він є
    customer_info = order.get("customer", {})
    card_id = order.get("payment", {}).get("card_id")

    print(f"Замовник: {customer_info}, \nНомер кредитної картки: {card_id}")

Замовлення, вартість яких перевищує 1500 одиниць:
Замовник: {'name': 'Andrii', 'surname': 'Rodinov', 'phones': [9876543, 1234567], 'address': 'PTI, Peremohy 37, Kyiv, UA'}, 
Номер кредитної картки: 12345678


In [52]:
# Вкажіть період дат
start_date = '2024-04-25'  # Початок періоду (рік, місяць, день)
end_date = '2024-05-25'    # Кінець періоду (рік, місяць, день)
# Вкажіть ObjectId товару, який потрібно видалити з замовлень
item_id_to_remove = item1["_id"]
query_filter = {
    "date": {"$gte": start_date, "$lte": end_date}
}

# Пошук замовлень, зроблених за цей період
found_orders = orders.find({"date": {"$gte": start_date, "$lte": end_date}})

# Виведення результатів

# Виконання запиту для видалення товару з замовлень
result = orders.update_many(
    query_filter,
    {"$pull": {"items_id": item_id_to_remove}}
)

# Вивід результатів
print(f"Кількість замовлень, з яких видалено товар: {result.modified_count}")

Кількість замовлень, з яких видалено товар: 2


Перейменуйте у всіх замовлення ім'я (прізвище) замовника

In [53]:
# Нова назва для замовника (наприклад, нове ім'я або прізвище)
new_name = "Andriy"  # Змінюйте на потрібне ім'я
new_surname = "Volovyk"  # Змінюйте на потрібне прізвище

# Виконання запиту для оновлення імені та/або прізвища у всіх замовленнях
result = orders.update_many(
    {},  # Порожній фільтр, який означає, що зміна стосуватиметься всіх замовлень
    {
        # Оновлення імені та прізвища замовника
        "$set": {
            "customer.name": new_name,  # Заміна імені на нове значення
            "customer.surname": new_surname  # Заміна прізвища на нове значення
        }
    }
)

# Вивід результатів
print(f"Кількість замовлень, у яких оновлено ім'я або прізвище: {result.modified_count}")

Кількість замовлень, у яких оновлено ім'я або прізвище: 2


Знайдіть замовлення зроблені одним замовником, і виведіть тільки інформацію про кастомера та товари у замовлені підставивши замість ObjectId("***") назви товарів та їх вартість (аналог join-а між таблицями orders та items).

In [57]:
# Вкажіть ім'я замовника, для якого хочете знайти замовлення
customer_name = "Andriy"

# Виконання операції агрегування
pipeline = [
    {
        # Фільтрація замовлень за ім'ям замовника
        "$match": {
            "customer.name": customer_name
        }
    },
    {
        # З'єднання з колекцією items за items_id
        "$lookup": {
            "from": "items",  # З якої колекції здійснюється з'єднання
            "localField": "items_id",  # Поле з ключами у замовленнях
            "foreignField": "_id",  # Поле з ключами у колекції items
            "as": "items_info"  # Як називати поле зі з'єднаними даними
        }
    },
    {
        # Вибірка тільки інформації про кастомера та товарів у замовленні
        "$project": {
            "customer": 1,  # Включаємо інформацію про кастомера
            "items_info.model": 1,  # Включаємо назви товарів
            "items_info.price": 1  # Включаємо ціни товарів
        }
    }
]

# Виконання операції агрегування та вивід результатів
result = orders.aggregate(pipeline)
print(f"Замовлення зроблені замовником {customer_name}:")
for order in result:
    customer_info = order.get("customer")
    items_info = order.get("items_info")

    print(f"Кастомер: {customer_info}")
    print("Товари у замовленні:")
    for item in items_info:
        name = item.get("model")
        price = item.get("price")
        print(f"- Назва: {name}, Ціна: {price}")
    print("-" * 30)

Замовлення зроблені замовником Andriy:
Кастомер: {'name': 'Andriy', 'surname': 'Volovyk', 'phones': [9876543, 1234567], 'address': 'PTI, Peremohy 37, Kyiv, UA'}
Товари у замовленні:
- Назва: Sony Bravia, Ціна: 1200
- Назва: OLED65, Ціна: 1200
------------------------------
Кастомер: {'name': 'Andriy', 'surname': 'Volovyk', 'phones': [2345678], 'address': 'PTI, Peremohy 37, Kyiv, UA'}
Товари у замовленні:
- Назва: Sony Bravia, Ціна: 1200
- Назва: Galaxy Watch, Ціна: 300
------------------------------


In [58]:
collection_name = 'reviews'
if collection_name in db.list_collection_names():
    db.drop_collection(collection_name)

# Виберіть розмір та максимальну кількість документів для колекції
# Розмір визначає максимальний об'єм у байтах, що може займати колекція
# max визначає максимальну кількість документів у колекції
db.create_collection(
    collection_name,
    capped=True,
    size=1024 * 10,  # Приклад: 10 кілобайт
    max=5  # Максимум 5 документів
)

# Отримання доступу до колекції
reviews = db[collection_name]

# Додавання кількох відгуків
for i in range(7):
    review = {
        "reviewer": f"Customer {i+1}",
        "rating": i % 5 + 1,
        "comment": f"Review comment {i+1}"
    }
    reviews.insert_one(review)
    print(f"Inserted review {i+1}")

# Перевірка вмісту колекції після додавання відгуків
print("\nReviews in the collection:")
for review in reviews.find():
    print(review)

Inserted review 1
Inserted review 2
Inserted review 3
Inserted review 4
Inserted review 5
Inserted review 6
Inserted review 7

Reviews in the collection:
{'_id': ObjectId('663357f3f9ec9c8290cd69d1'), 'reviewer': 'Customer 3', 'rating': 3, 'comment': 'Review comment 3'}
{'_id': ObjectId('663357f3f9ec9c8290cd69d2'), 'reviewer': 'Customer 4', 'rating': 4, 'comment': 'Review comment 4'}
{'_id': ObjectId('663357f3f9ec9c8290cd69d3'), 'reviewer': 'Customer 5', 'rating': 5, 'comment': 'Review comment 5'}
{'_id': ObjectId('663357f3f9ec9c8290cd69d4'), 'reviewer': 'Customer 6', 'rating': 1, 'comment': 'Review comment 6'}
{'_id': ObjectId('663357f3f9ec9c8290cd69d5'), 'reviewer': 'Customer 7', 'rating': 2, 'comment': 'Review comment 7'}
